# Side effects Data

To run this you first have to run preprocessing/01_classifying text lines and also preprocessing/02_ms-classification and preprocessing 04/llama-finetune. This ensures that you have the same data.

In [85]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

import pandas as pd

from datasets import Dataset

import json

In [86]:
df = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"text-finetune/kisim_diagnoses_combined.csv")

In [87]:
keywords = ["Nebenwirkung","Unverträglichkeit", "Begleiterscheinung", "Begleitsymptom", "Begleitphänomen"]

# Filter out all the entries that contain any of the keywords in the text
df_se = df[df["text"].str.contains("|".join(keywords), case=False, na=False)]

In [88]:
# Print the lines that contain the keyword
for text in df_se["text"]:
    text = text.splitlines()
    for line in text:
        if any(keyword in line for keyword in keywords):
            print(line)
            print("-" * 50)

INDENT 12/2014 - 08/2015 Tecfidera, sistiert bei gastrointestinalen Nebenwirkungen
--------------------------------------------------
INDENT 06/2016 - 07/2016 Tecfidera, abgesetzt wegen Unverträglichkeit mit Exanthem, Schwindel, Tremor, Gangunsicherheit
--------------------------------------------------
INDENT 11/97 - 02/05: Rebif (Stopp wegen Nebenwirkungen)
--------------------------------------------------
INDENT 08/08 - 10/09: Rebif 3x44 ug (erneuter Beginn; wg. Nebenwirkungen abgesetzt) 
--------------------------------------------------
Nicht näher bezeichnete unerwünschte Nebenwirkung eines Arzneimittels oder einer Droge
--------------------------------------------------
INDENT 11/2014 - 08/2017 Tecfidera, keine relevanten Nebenwirkungen
--------------------------------------------------
INDENT 06/2016 - 07/2016 Tecfidera, abgesetzt wegen Unverträglichkeit mit Exanthem, Schwindel, Tremor, Gangunsicherheit
--------------------------------------------------
INDENT St.n. Therapie m

In [89]:
len(df_se)

412

- We have 1600 texts that should contain something. Will sample 100 of them

In [90]:
# Sample 100 entries
df_se_sample = df_se.sample(100, random_state=42)

In [91]:
for text in df_se_sample["text"]:
    text = text.splitlines()
    for line in text:
        if any(keyword in line for keyword in keywords):
            print(line)
            print("-" * 50)
            print(text)
            print("=" * 50)

INDENT kurzfristige Therapie mit Tecfidera 10/2015, wegen fehlender Wirkung und Nebenwirkungen abgesetzt
--------------------------------------------------
['1. Multiple Sklerose a.e. primär progredienter Verlauf, ES und ED 07/2006, EDSS 7.5', 'klinisch:', 'INDENT aktuell: bein- und rechtsbetonte spastische Tetraparese ', 'INDENT Verlauf: EM 2006 mit subakut aufgetretener Beinparese rechts, progrediente Gangstörung mit Rollstuhlpflichtigkeit seit 2014, keine eigentliche schubverdächtigen Ereignisse, keine sonstigen Symptome', 'INDENT diagnostisch:', 'INDENT Serologien 06/2006: RF, ANA, ANCA, HIV, Lues TPPA, Borrelien, Lyme negativ', 'INDENT Liquor 06/2006: Eiweiss 394 mg/l, Zellzahl 2.0/ µl, OKB positiv', 'INDENT anti-NMO AK negativ 2013', 'INDENT MRI Schädel und HWS 09/2015 (extern): neuer florider Herd im Pons rechts, sonst unverändert nicht floride Herde im Gehirn sowie im zervikalen und oberen thorakalen Myelon', 'INDENT cMRI 07/2016: Multiple infra- und supratentorielle MS-Plaques

In [92]:
# Create HF dataset
os.makedirs(paths.DATA_PATH_PREPROCESSED/"side-effects", exist_ok=True)
df_se_sample.rename(columns={"research_id": "rid"}, inplace=True)
df_se_sample = df_se_sample[["rid", "text"]]
df_se_sample.to_csv(paths.DATA_PATH_PREPROCESSED/"side-effects/kisim_diagnoses_combined_se_sample.csv", index=False)
ds_se = Dataset.from_pandas(df_se_sample, preserve_index=False)

In [93]:
# Save the dataset
ds_se.save_to_disk(paths.DATA_PATH_PREPROCESSED/"side-effects/kisim_diagnoses_combined_se_sample")

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

# Prompting

In [94]:
# Prompt
task_instruction ="""Your task is to extract medications that showed adverse side effects from a medical report. 
The input for this task is a medical report in text form, and the output should be a complete list of dictionaries (one per medication) with the following keys:

- "medication" (str): the name of the medication
- "side_effect" (str): the side effect of the medication

The output format should look like this:
[
    {"medication": "MedicationName1", "side_effect": "SideEffect1"},
    {"medication": "MedicationName2", "side_effect": "SideEffect2"},
    ...
]

- The "MedicationName" can consist of multiple words with whitespace and should be returned as a single string. If you don't find any mentions of medications or drugs, it should be represented as "unknown".
- The "SideEffect" describes the specific adverse event that was caused by the medication. Sometimes it is not explicitly mentioned; in this case, it should be represented as "unknown".

Only medications that showed adverse side effects should be extracted. If a medication is mentioned but no side effect is mentioned, it should not be included in the output.

"""

system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don’t know the answer to a question, please don’t share false information.
"""

with open(paths.DATA_PATH_PREPROCESSED/"side-effects/task_instruction.txt", "w") as file:
    file.write(task_instruction)
with open(paths.DATA_PATH_PREPROCESSED/"side-effects/system_instruction.txt", "w") as file:
    file.write(system_prompt)

In [95]:
examples = [
    {"text": "kurzfristige Therapie mit Medrol über 3 Wochen, wegen Nebenwirkungen abgesetzt", "labels" : '[{"medication": "Medrol", "side_effect": "unknown"}]'},
    {"text": "Sertralin wegen Unverträglichkeit (Ausschläge, Durchfall, Gastrointestale Beschwerden) abgesetzt", "labels" : '[{"medication": "Sertralin", "side_effect": "Ausschläge, Durchfall, Gastrointestale Beschwerden"}]'},
    {"text": "Absetzen von Metoprolol aufgrund von Müdigkeit und Schwindel. Wegen Bauchweh absetzen von Aspirin", "labels" : '[{"medication": "Metoprolol", "side_effect": "Müdigkeit und Schwindel"}, {"medication": "Aspirin", "side_effect": "Bauchweh"}]'},
    {"text" : "Aufgrund von starken Nebenwirkungen wurde das Medikament abgesetzt", "labels" : '[{"medication": "unknown", "side_effect": "unknown"}]'},
    {"text": "Unverträglichkeit von Ibuprofen, deshalb abgesetzt", "labels" : '[{"medication": "Ibuprofen", "side_effect": "unknown"}]'},
    {"text": "Patient berichtet über Hitzewallungen und Übelkeit nach Einnahme von Paracetamol", "labels": '[{"medication": "Paracetamol", "side_effect": "Hitzewallungen und Übelkeit"}]'},
    {"text": "Durchfall und Kopfschmerzen nach Beginn von Amoxicillin. Absetzen des Antibiotikums", "labels": '[{"medication": "Amoxicillin", "side_effect": "Durchfall und Kopfschmerzen"}]'},
    {"text": "Patient klagt über Schwindel und Hautausschlag nach Einnahme von Doxycyclin", "labels": '[{"medication": "Doxycyclin", "side_effect": "Schwindel und Hautausschlag"}]'},
    {"text": "Verstärkte Magenbeschwerden und Schlaflosigkeit nach Einnahme von Pantoprazol", "labels": '[{"medication": "Pantoprazol", "side_effect": "Verstärkte Magenbeschwerden und Schlaflosigkeit"}]'},
    {"text": "Unruhe und Herzklopfen nach Beginn von Venlafaxin. Abgesetzt aufgrund der Nebenwirkungen", "labels": '[{"medication": "Venlafaxin", "side_effect": "Unruhe und Herzklopfen"}]'},
]

# Save dictionary
with open(paths.DATA_PATH_PREPROCESSED/"side-effects/examples.json", "w") as file:
    json.dump(examples, file, indent=4)